<a href="https://colab.research.google.com/github/JeongJeong-code/P6_market_place/blob/main/text_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys
import numpy as np
import matplotlib.pyplot as plt
from enum import Enum
from google.colab import files
import pandas as pd
import seaborn as sns
import os.path
import missingno as mi
import re
import sklearn as sk
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics.cluster import adjusted_rand_score
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.manifold import TSNE
from sklearn.metrics.cluster import homogeneity_score
from sklearn.decomposition import KernelPCA
from sklearn.cluster import (AgglomerativeClustering,
                             KMeans, DBSCAN, SpectralClustering)
from sklearn.metrics import davies_bouldin_score, silhouette_score
! {sys.executable} -m pip install ipynb

import cv2
from tqdm import tqdm

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [30]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [2]:
url = 'https://raw.githubusercontent.com/JeongJeong-code/P6_market_place/main/flipkart_com-ecommerce_sample_1050.csv'
df_raw= pd.read_csv(url)

In [3]:
desc_token = df_raw['description'].copy()

In [12]:
for i in range(len(desc_token)):
  desc_token[i] = nltk.word_tokenize(df_raw.description[i])

In [21]:
desc_token[0]

['Key',
 'Features',
 'of',
 'Elegance',
 'Polyester',
 'Multicolor',
 'Abstract',
 'Eyelet',
 'Door',
 'Curtain',
 'Floral',
 'Curtain',
 ',',
 'Elegance',
 'Polyester',
 'Multicolor',
 'Abstract',
 'Eyelet',
 'Door',
 'Curtain',
 '(',
 '213',
 'cm',
 'in',
 'Height',
 ',',
 'Pack',
 'of',
 '2',
 ')',
 'Price',
 ':',
 'Rs',
 '.',
 '899',
 'This',
 'curtain',
 'enhances',
 'the',
 'look',
 'of',
 'the',
 'interiors.This',
 'curtain',
 'is',
 'made',
 'from',
 '100',
 '%',
 'high',
 'quality',
 'polyester',
 'fabric.It',
 'features',
 'an',
 'eyelet',
 'style',
 'stitch',
 'with',
 'Metal',
 'Ring.It',
 'makes',
 'the',
 'room',
 'environment',
 'romantic',
 'and',
 'loving.This',
 'curtain',
 'is',
 'ant-',
 'wrinkle',
 'and',
 'anti',
 'shrinkage',
 'and',
 'have',
 'elegant',
 'apparance.Give',
 'your',
 'home',
 'a',
 'bright',
 'and',
 'modernistic',
 'appeal',
 'with',
 'these',
 'designs',
 '.',
 'The',
 'surreal',
 'attention',
 'is',
 'sure',
 'to',
 'steal',
 'hearts',
 '.',
 

In [50]:
import string
from nltk.stem import PorterStemmer
token_stop = desc_token.copy()
stemmed_desc= desc_token.copy()
wordnet_lemmatizer = nltk.WordNetLemmatizer()
lemmatize_words = np.vectorize(wordnet_lemmatizer.lemmatize)
porter= PorterStemmer()# vectorizing function to able to call on list of tokens
stem_words = np.vectorize(porter.stem)
for i in range(len(desc_token)):
  nltk_stop_words = nltk.corpus.stopwords.words('english')
  token_stop[i] = [t for t in desc_token[i] if t not in nltk_stop_words]
  token_stop[i] = [t for t in token_stop[i] if t not in string.punctuation]
  stemmed_text = stem_words(token_stop[i])
  stemmed_desc[i] = stemmed_text
  #display(f"nltk text without stop words: {text_without_stop_words}")

In [53]:
list(nltk.bigrams(stemmed_desc[0]))

[('key', 'featur'),
 ('featur', 'eleg'),
 ('eleg', 'polyest'),
 ('polyest', 'multicolor'),
 ('multicolor', 'abstract'),
 ('abstract', 'eyelet'),
 ('eyelet', 'door'),
 ('door', 'curtain'),
 ('curtain', 'floral'),
 ('floral', 'curtain'),
 ('curtain', 'eleg'),
 ('eleg', 'polyest'),
 ('polyest', 'multicolor'),
 ('multicolor', 'abstract'),
 ('abstract', 'eyelet'),
 ('eyelet', 'door'),
 ('door', 'curtain'),
 ('curtain', '213'),
 ('213', 'cm'),
 ('cm', 'height'),
 ('height', 'pack'),
 ('pack', '2'),
 ('2', 'price'),
 ('price', 'Rs'),
 ('Rs', '899'),
 ('899', 'thi'),
 ('thi', 'curtain'),
 ('curtain', 'enhanc'),
 ('enhanc', 'look'),
 ('look', 'interiors.thi'),
 ('interiors.thi', 'curtain'),
 ('curtain', 'made'),
 ('made', '100'),
 ('100', 'high'),
 ('high', 'qualiti'),
 ('qualiti', 'polyest'),
 ('polyest', 'fabric.it'),
 ('fabric.it', 'featur'),
 ('featur', 'eyelet'),
 ('eyelet', 'style'),
 ('style', 'stitch'),
 ('stitch', 'metal'),
 ('metal', 'ring.it'),
 ('ring.it', 'make'),
 ('make', 'room')

In [51]:
stemmed_desc[0:5]

0    [key, featur, eleg, polyest, multicolor, abstr...
1    [specif, sathiya, cotton, bath, towel, 3, bath...
2    [key, featur, eurospa, cotton, terri, face, to...
3    [key, featur, santosh, royal, fashion, cotton,...
4    [key, featur, jaipur, print, cotton, floral, k...
Name: description, dtype: object